# SCGNN Test - Trafik - No PCA

In [1]:
import pandas as pd
import sys
from pathlib import Path
sys.path.append('../..')
from src import config
from src.utils.dataset import TrafficDataset
from src.utils.model import TrafficPredictor
from torch.utils.data import DataLoader
import torch

In [2]:
df = pd.read_parquet(config.DATA_INTERIM / 'data.parquet')
df = df.fillna(0)
X_STEP, Y_STEP = 2, 1
data_array = df.values

In [3]:
# PCA kullanılmıyor
dataset = TrafficDataset(data_array, X_STEP, Y_STEP)
sensors = dataset.sensors
edges = []
for i in range(sensors - 1):
    edges.append([i, i + 1])
    edges.append([i + 1, i])
edge_index = torch.tensor(edges, dtype=torch.long).t()
predictor = TrafficPredictor('scgnn', sensors, X_STEP, Y_STEP, edge_index=edge_index, pca_model=dataset.pca if False else None)
predictor.load_model(config.PROJECT_ROOT.joinpath('models/scgnn_trafik_nopca.pt'))
loader = DataLoader(dataset, batch_size=10, num_workers=8)
preds = predictor.inference(loader)
display(pd.DataFrame(preds.squeeze()).head().iloc[:, :10])


,0,1,2,3,4,5,6,7,8,9
0,33.226337,34.541969,28.290535,28.206543,37.188896,44.237839,46.122341,39.917522,40.254150,44.460144
1,58.429096,14.853082,19.460476,31.390574,31.390574,45.047031,66.929359,25.982868,30.464512,41.005524
2,59.669537,14.641556,27.874962,42.308838,41.782932,53.713028,74.292038,32.105103,35.472061,48.328217
3,63.036499,15.967768,38.438873,54.261833,52.746994,62.973621,88.663033,48.368202,52.449696,69.513092
4,69.221603,22.341509,39.113426,55.988197,57.691666,73.491737,102.971138,65.328743,70.736450,92.555901
